In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data = load_iris()

In [ ]:
X = data["data"] 
y = data["target"] 

In [ ]:
df = pd.DataFrame(X, columns=data["feature_names"])

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
class ParzenWindow:
    
    def __init__(self, radius):
        self.radius = radius
        self.X = None
        self.y = None
        self.n_classes = None
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.n_classes = np.unique(y).size
        
    def kernel(self, r):
        return (1/np.sqrt(2 * np.pi)) * (np.exp((-0.5) * (r ** 2)))
    
    def predict_proba(self, X):
        
        distances = cdist(X, self.X)
        results = []
        for i in range(distances.shape[0]):
            dist_row = distances[i]
            indexes = np.where(dist_row < self.radius)
            distances_lower = dist_row[indexes]
            r = distances_lower / self.radius
            k = self.kernel(r)
            labels = self.y[indexes]
            probas = np.zeros(self.n_classes)
            for c in range(self.n_classes):
                if (labels == c).sum() > 0:
                    probas[c] = k[labels == c].sum()
            probas /= probas.sum()
            results.append(probas)
        return results
    
    def predict(self, X):
        probas = self.predict_proba(X)
        return np.argmax(probas, axis=1)

In [ ]:
class KNeighborsClassifier:
    
    def __init__(self, n_closest):
        self.n_closest = n_closest + 1
        self.X = None
        self.y = None
        self.n_classes = None
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.n_classes = np.unique(y).size

    def kernel(self, r):
        return (1/np.sqrt(2 * np.pi)) * (np.exp((-0.5) * (r ** 2)))

    def predict_proba(self, X):
        
        distances = cdist(X, self.X)
        results = []
        max_radius = np.sort(distances, axis=1)[:,self.n_closest]

        for i in range(distances.shape[0]):
            radius = max_radius[i]
            dist_row = distances[i]
            indexes = np.where(dist_row < radius)
            distances_lower = dist_row[indexes]
            r = distances_lower / radius
            k = self.kernel(r)
            labels = self.y[indexes]
            probas = np.zeros(self.n_classes)
            for c in range(self.n_classes):
                if (labels == c).sum() > 0:
                    probas[c] = k[labels == c].sum()
            probas /= probas.sum()
            results.append(probas)
        return results
    
    def predict(self, X):
        probas = self.predict_proba(X)
        return np.argmax(probas, axis=1)

In [ ]:
knn = KNeighborsClassifier(3)
knn.fit(train_X, train_y)
pred = knn.predict(test_X)
accuracy_score(test_y, pred)

1.0